In [4]:
import obspy
import os
import csv

cwd = os.getcwd()
DATA_PATH = '/kik_knet_data'
ALL_DATA_PATH = f'{DATA_PATH}/all_data'
RAW_DATA_PATH = f'{ALL_DATA_PATH}/selecteddata'
k = cwd + RAW_DATA_PATH + '/*/*.*'
print(k)
# Format will be detected automatically.
st = obspy.read(k, format="knet")
# print(st)
# Many formats can also be written out - just use the module name.
# st.write(cwd + '/hey', format="pickle")
# # import pandas as pd
# # unpickled_df = pd.read_pickle("./hey")
# # unpickled_df

# import pickle
# data = pickle.load(open('./hey', "rb"))
# print(data[0], type(data))
# print(dir(st[0]))
# st.plot()
# print(len(st))

/home/adebowale/code/proj/kik_knet_data/all_data/selecteddata/*/*.*
960 Trace(s) in Stream:

BO.AKTH04..EW1 | 2008-06-13T23:43:34.000000Z - 2008-06-13T23:48:33.990000Z | 100.0 Hz, 30000 samples
...
(958 other traces)
...
BO... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:00:00.000000Z | 1.0 Hz, 0 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]


In [5]:
print(st)

960 Trace(s) in Stream:

BO.AKTH04..EW1 | 2008-06-13T23:43:34.000000Z - 2008-06-13T23:48:33.990000Z | 100.0 Hz, 30000 samples
...
(958 other traces)
...
BO... | 1970-01-01T00:00:00.000000Z - 1970-01-01T00:00:00.000000Z | 1.0 Hz, 0 samples

[Use "print(Stream.__str__(extended=True))" to print all Traces]


In [6]:
tr = st[500]

In [8]:
tr.stats

         network: BO
         station: KMMH02
        location: 
         channel: UD1
       starttime: 2016-04-15T16:24:59.000000Z
         endtime: 2016-04-15T16:29:58.990000Z
   sampling_rate: 100.0
           delta: 0.01
            npts: 30000
           calib: 4.764783388733394e-06
         _format: KNET
            knet: AttribDict({'evot': UTCDateTime(2016, 4, 15, 16, 25), 'evla': 32.753, 'evlo': 130.762, 'evdp': 12.0, 'mag': 7.3, 'stla': 33.122, 'stlo': 131.0629, 'stel': 306.45, 'duration': 300.0, 'accmax': 71.922, 'last correction': UTCDateTime(2016, 4, 15, 16, 24, 59)})

In [9]:
tr.data

array([16758., 16759., 16759., ..., 16571., 16690., 16940.])